In [7]:
!python -V

Python 3.12.3


In [1]:
import math
import os.path
import re
import sys
from os import path

from loguru import logger

from app.config import config
from app.models import const
from app.models.schema import VideoParams, VideoConcatMode
from app.services import llm, material, voice, video, subtitle
from app.services import state as sm
from app.utils import utils

2024-05-10 20:03:04.042 | INFO     | app.config.config:load_config:22 - load config from file: D:\work\MoneyPrinterTurbo/config.toml
2024-05-10 20:03:04.048 | INFO     | app.config.config:<module>:70 - MoneyPrinterTurbo v1.1.5


In [2]:
params_dict = {
        "video_subject": "怎么快速赚到100万",
        "video_aspect": "9:16",
        "voice_name": "zh-CN-XiaoxiaoNeural",
        "enable_bgm": False,
        "font_name": "STHeitiMedium 黑体-中",
        "text_color": "#FFFFFF",
        "font_size": 60,
        "stroke_color": "#000000",
        "stroke_width": 1.5
    }

params = VideoParams(**params_dict)
task_id = 'task_id_test_123'

In [3]:
params.video_subject


'怎么快速赚到100万'

In [4]:
config.app.get("openai_model_name", "openai")

'gpt-3.5-turbo'

In [5]:
video_subject = params.video_subject
voice_name = voice.parse_voice_name(params.voice_name)
paragraph_number = params.paragraph_number
n_threads = params.n_threads
max_clip_duration = params.video_clip_duration
logger.info("\n\n## generating video script")
video_script = params.video_script.strip()
if not video_script:
    video_script = llm.generate_script(video_subject=video_subject, language=params.video_language,
                                       paragraph_number=paragraph_number)
else:
    logger.debug(f"video script: \n{video_script}")
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=10)

2024-05-10 20:03:14 | INFO | "./app\services\llm.py:215": generate_script - subject: 怎么快速赚到100万
2024-05-10 20:03:14 | INFO | "./app\services\llm.py:16": _generate_response - llm provider: openai


--- Logging error in Loguru Handler #1 ---
Record was: {'elapsed': datetime.timedelta(seconds=10, microseconds=492275), 'exception': None, 'extra': {}, 'file': (name='2255889761.py', path='C:\\Users\\roperluo\\AppData\\Local\\Temp\\ipykernel_42196\\2255889761.py'), 'function': '<module>', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 6, 'message': '\n\n## generating video script', 'module': '2255889761', 'name': '__main__', 'process': (id=42196, name='MainProcess'), 'thread': (id=43444, name='MainThread'), 'time': datetime(2024, 5, 10, 20, 3, 14, 523626, tzinfo=datetime.timezone(datetime.timedelta(seconds=28800), '中国标准时间'))}
Traceback (most recent call last):
  File "D:\software\miniforge3\envs\video-gen\Lib\site-packages\loguru\_handler.py", line 137, in emit
    dynamic_format = self._formatter(record)
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\work\MoneyPrinterTurbo\app\config\__init__.py", line 19, in format_record
    relative_path = os.path.relpath(file_path, roo

2024-05-10 20:03:18 | SUCCESS | "./app\services\llm.py:248": generate_script - completed: 
本视频将教你如何快速赚到100万。首先，要制定一个详细的理财计划，包括存款、投资和理财产品的选择。其次，要提高自己的收入水平，可以考虑副业、加班或职业转换。最后，要控制消费习惯，避免过度消费和无谓支出。践行这些方法，相信你很快可以赚到100万。


In [6]:
logger.info("\n\n## generating video terms")
video_terms = params.video_terms
if not video_terms:
    video_terms = llm.generate_terms(video_subject=video_subject, video_script=video_script, amount=5)
else:
    if isinstance(video_terms, str):
        video_terms = [term.strip() for term in re.split(r'[,，]', video_terms)]
    elif isinstance(video_terms, list):
        video_terms = [term.strip() for term in video_terms]
    else:
        raise ValueError("video_terms must be a string or a list of strings.")
    logger.debug(f"video terms: {utils.to_json(video_terms)}")
script_file = path.join(utils.task_dir(task_id), f"script.json")
script_data = {
    "script": video_script,
    "search_terms": video_terms,
    "params": params,
}
with open(script_file, "w", encoding="utf-8") as f:
    f.write(utils.to_json(script_data))
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=20)

2024-05-10 20:03:18 | INFO | "./app\services\llm.py:279": generate_terms - subject: 怎么快速赚到100万
2024-05-10 20:03:18 | INFO | "./app\services\llm.py:16": _generate_response - llm provider: openai


--- Logging error in Loguru Handler #1 ---
Record was: {'elapsed': datetime.timedelta(seconds=14, microseconds=357920), 'exception': None, 'extra': {}, 'file': (name='447892559.py', path='C:\\Users\\roperluo\\AppData\\Local\\Temp\\ipykernel_42196\\447892559.py'), 'function': '<module>', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 1, 'message': '\n\n## generating video terms', 'module': '447892559', 'name': '__main__', 'process': (id=42196, name='MainProcess'), 'thread': (id=43444, name='MainThread'), 'time': datetime(2024, 5, 10, 20, 3, 18, 389271, tzinfo=datetime.timezone(datetime.timedelta(seconds=28800), '中国标准时间'))}
Traceback (most recent call last):
  File "D:\software\miniforge3\envs\video-gen\Lib\site-packages\loguru\_handler.py", line 137, in emit
    dynamic_format = self._formatter(record)
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\work\MoneyPrinterTurbo\app\config\__init__.py", line 19, in format_record
    relative_path = os.path.relpath(file_path, root_di

2024-05-10 20:03:19 | SUCCESS | "./app\services\llm.py:300": generate_terms - completed: 
['how to make money fast', 'financial planning tips', 'increase income strategies', 'investment options guide', 'control spending habits']


In [14]:
import asyncio
import os
import re
from datetime import datetime
from xml.sax.saxutils import unescape
from edge_tts.submaker import mktimestamp
from loguru import logger
from edge_tts import submaker, SubMaker
import edge_tts
from moviepy.video.tools import subtitles

from app.config import config
from app.utils import utils
import nest_asyncio

nest_asyncio.apply()
def azure_tts_v1(text: str, voice_name: str, voice_file: str) -> [SubMaker, None]:
    text = text.strip()
    for i in range(3):
        # try:
            logger.info(f"start, voice name: {voice_name}, try: {i + 1}")

            async def _do() -> SubMaker:
                communicate = edge_tts.Communicate(text, voice_name)
                sub_maker = edge_tts.SubMaker()
                with open(voice_file, "wb") as file:
                    async for chunk in communicate.stream():
                        if chunk["type"] == "audio":
                            file.write(chunk["data"])
                        elif chunk["type"] == "WordBoundary":
                            sub_maker.create_sub((chunk["offset"], chunk["duration"]), chunk["text"])
                return sub_maker

            # sub_maker = asyncio.get_event_loop().run_until_complete(_do())
            sub_maker = asyncio.run(_do())
            if not sub_maker or not sub_maker.subs:
                logger.warning(f"failed, sub_maker is None or sub_maker.subs is None")
                continue

            logger.info(f"completed, output file: {voice_file}")
            return sub_maker
        # except Exception as e:
        #     logger.error(f"failed, error: {str(e)}")
    return None

In [15]:

audio_file = path.join(utils.task_dir(task_id), f"audio.mp3")
sub_maker = azure_tts_v1(text=video_script, voice_name=voice_name, voice_file=audio_file)
if sub_maker is None:
    sm.state.update_task(task_id, state=const.TASK_STATE_FAILED)
    logger.error(
        """failed to generate audio:
1. check if the language of the voice matches the language of the video script.
2. check if the network is available. If you are in China, it is recommended to use a VPN and enable the global traffic mode.
    """.strip()
    )
    
audio_duration = voice.get_audio_duration(sub_maker)
audio_duration = math.ceil(audio_duration)
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=30)

--- Logging error in Loguru Handler #1 ---
Record was: {'elapsed': datetime.timedelta(seconds=503, microseconds=593448), 'exception': None, 'extra': {}, 'file': (name='3954587724.py', path='C:\\Users\\roperluo\\AppData\\Local\\Temp\\ipykernel_42196\\3954587724.py'), 'function': 'azure_tts_v1', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 21, 'message': 'start, voice name: zh-CN-XiaoxiaoNeural, try: 1', 'module': '3954587724', 'name': '__main__', 'process': (id=42196, name='MainProcess'), 'thread': (id=43444, name='MainThread'), 'time': datetime(2024, 5, 10, 20, 11, 27, 624799, tzinfo=datetime.timezone(datetime.timedelta(seconds=28800), '中国标准时间'))}
Traceback (most recent call last):
  File "D:\software\miniforge3\envs\video-gen\Lib\site-packages\loguru\_handler.py", line 137, in emit
    dynamic_format = self._formatter(record)
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\work\MoneyPrinterTurbo\app\config\__init__.py", line 19, in format_record
    relative_path = os.pat

In [17]:
subtitle_path = ""
if params.subtitle_enabled:
    subtitle_path = path.join(utils.task_dir(task_id), f"subtitle.srt")
    subtitle_provider = config.app.get("subtitle_provider", "").strip().lower()
    logger.info(f"\n\n## generating subtitle, provider: {subtitle_provider}")
    subtitle_fallback = False
    if subtitle_provider == "edge":
        voice.create_subtitle(text=video_script, sub_maker=sub_maker, subtitle_file=subtitle_path)
        if not os.path.exists(subtitle_path):
            subtitle_fallback = True
            logger.warning("subtitle file not found, fallback to whisper")
    if subtitle_provider == "whisper" or subtitle_fallback:
        subtitle.create(audio_file=audio_file, subtitle_file=subtitle_path)
        logger.info("\n\n## correcting subtitle")
        subtitle.correct(subtitle_file=subtitle_path, video_script=video_script)
    subtitle_lines = subtitle.file_to_subtitles(subtitle_path)
    if not subtitle_lines:
        logger.warning(f"subtitle file is invalid: {subtitle_path}")
        subtitle_path = ""

2024-05-10 20:15:34 | INFO | "./app\services\voice.py:1232": create_subtitle - completed, subtitle file created: D:\work\MoneyPrinterTurbo\storage\tasks\task_id_test_123\subtitle.srt, duration: 26.475


--- Logging error in Loguru Handler #1 ---
Record was: {'elapsed': datetime.timedelta(seconds=750, microseconds=777049), 'exception': None, 'extra': {}, 'file': (name='3129935629.py', path='C:\\Users\\roperluo\\AppData\\Local\\Temp\\ipykernel_42196\\3129935629.py'), 'function': '<module>', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 5, 'message': '\n\n## generating subtitle, provider: edge', 'module': '3129935629', 'name': '__main__', 'process': (id=42196, name='MainProcess'), 'thread': (id=43444, name='MainThread'), 'time': datetime(2024, 5, 10, 20, 15, 34, 808400, tzinfo=datetime.timezone(datetime.timedelta(seconds=28800), '中国标准时间'))}
Traceback (most recent call last):
  File "D:\software\miniforge3\envs\video-gen\Lib\site-packages\loguru\_handler.py", line 137, in emit
    dynamic_format = self._formatter(record)
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\work\MoneyPrinterTurbo\app\config\__init__.py", line 19, in format_record
    relative_path = os.path.relpath(

In [18]:
sub_maker

In [19]:
sub_maker.subs

['本',
 '视频',
 '将',
 '教',
 '你',
 '如何',
 '快速',
 '赚',
 '到',
 '100万',
 '首先',
 '要',
 '制定',
 '一个',
 '详细',
 '的',
 '理财',
 '计划',
 '包括',
 '存款',
 '投资',
 '和',
 '理财',
 '产品',
 '的',
 '选择',
 '其次',
 '要',
 '提高',
 '自己',
 '的',
 '收入',
 '水平',
 '可以',
 '考虑',
 '副业',
 '加班',
 '或',
 '职业',
 '转换',
 '最后',
 '要',
 '控制',
 '消费',
 '习惯',
 '避免',
 '过度',
 '消费',
 '和',
 '无谓',
 '支出',
 '践行',
 '这些',
 '方法',
 '相信',
 '你',
 '很快',
 '可以',
 '赚',
 '到',
 '100万']

In [20]:
sub_maker.offset

[(1000000, 2750000),
 (2750000, 8375000),
 (8625000, 10500000),
 (10625000, 12500000),
 (12500000, 14000000),
 (14000000, 17375000),
 (17500000, 23125000),
 (24125000, 25875000),
 (25875000, 27000000),
 (27250000, 33875000),
 (39500000, 45250000),
 (48125000, 49625000),
 (49750000, 52875000),
 (52875000, 55000000),
 (55250000, 59500000),
 (59500000, 60375000),
 (60625000, 63875000),
 (63875000, 68750000),
 (71625000, 74875000),
 (75125000, 81250000),
 (83000000, 88750000),
 (89750000, 91500000),
 (91625000, 95125000),
 (95125000, 98750000),
 (98750000, 99625000),
 (99750000, 105250000),
 (111000000, 116250000),
 (119250000, 121000000),
 (121125000, 124875000),
 (124875000, 127750000),
 (127750000, 128625000),
 (128875000, 132000000),
 (132000000, 137750000),
 (139625000, 142125000),
 (142125000, 145500000),
 (145750000, 150625000),
 (152750000, 157875000),
 (158875000, 161000000),
 (161125000, 164500000),
 (164500000, 170000000),
 (175625000, 180750000),
 (183250000, 185000000),
 (1850

In [21]:
print(len(sub_maker.subs), len(sub_maker.offset))

61 61
